##  Solution for Egen questions

###  * Use case #2

* Import necessary libraries

In [1]:
import pandas as pd
import csv
import warnings
from collections import Counter 
warnings.filterwarnings("ignore")

In [2]:
val_ds=pd.read_csv("/Users/sandipayannandi/Downloads/egen/Validation_DataSet.csv")

In [3]:
val_ds[1:10]

,CONTROL,AGE1,METRO3,REGION,LMED,FMR,L30,L50,L80,IPOV,...,FMTINCRELFMRCAT,FMTCOST06RELAMICAT,FMTCOST08RELAMICAT,FMTCOST12RELAMICAT,FMTCOSTMEDRELAMICAT,FMTINCRELAMICAT,FMTASSISTED,FMTBURDEN,FMTREGION,FMTSTATUS
1,100006110249',50,'5','3',55846,1100,17165,28604,45744,24218,...,'3 GT FMR','4 60 - 80% AMI','4 60 - 80% AMI','6 100 - 120% AMI','4 60 - 80% AMI','7 120% AMI +','.','1 Less than 30%','-5','-5'
2,'100006370140',53,'5','3',55846,1100,13750,22897,36614,15470,...,'2 50.1 - 100% FMR','4 60 - 80% AMI','5 80 - 100% AMI','6 100 - 120% AMI','4 60 - 80% AMI','4 60 - 80% AMI','.','3 50% or More','-5','-5'
3,'100006520140',67,'5','3',55846,949,13750,22897,36614,13964,...,'2 50.1 - 100% FMR','6 100 - 120% AMI','7 120% AMI +','7 120% AMI +','5 80 - 100% AMI','4 60 - 80% AMI','.','1 Less than 30%','-5','-5'
4,'100007130148',26,'1','3',60991,737,14801,24628,39421,15492,...,'3 GT FMR','3 50 - 60% AMI','3 50 - 60% AMI','3 50 - 60% AMI','3 50 - 60% AMI','7 120% AMI +','0 Not Assisted','1 Less than 30%','-5','-5'
5,'100007390148',56,'2','3',62066,657,13170,21924,35073,12005,...,'2 50.1 - 100% FMR','3 50 - 60% AMI','3 50 - 60% AMI','3 50 - 60% AMI','3 50 - 60% AMI','2 30 - 50% AMI','1 Assisted','3 50% or More','-5','-5'
6,'100007540148',50,'1','3',60991,988,16646,27713,44340,18050,...,'3 GT FMR','7 120% AMI +','7 120% AMI +','7 120% AMI +','6 100 - 120% AMI','7 120% AMI +','.','1 Less than 30%','-5','-5'
7,'100008700141',26,'4','4',52322,773,13489,22471,35929,15992,...,'3 GT FMR','4 60 - 80% AMI','4 60 - 80% AMI','4 60 - 80% AMI','4 60 - 80% AMI','4 60 - 80% AMI','0 Not Assisted','2 30% to 50%','West','-5'
8,'100008960141',60,'5','4',50296,1125,13115,21859,34939,15452,...,'3 GT FMR','6 100 - 120% AMI','6 100 - 120% AMI','7 120% AMI +','5 80 - 100% AMI','7 120% AMI +','.','1 Less than 30%','West','-5'
9,'100009170148',26,'4','2',63221,552,13338,22199,35501,12005,...,'3 GT FMR','5 80 - 100% AMI','5 80 - 100% AMI','5 80 - 100% AMI','5 80 - 100% AMI','6 100 - 120% AMI','0 Not Assisted','1 Less than 30%','-5','-5'


* Filter records based on below criteria and send filtered records to a file

Create a function which will check for the below logics and if there is a violation then the same records will be moved to the failed_records.csv file.

1.	STRUCTURETYPE should be Positive and Numeric 
2.	CONTROL should Unique   (* Move all duplicates to the failed_records)
3.	AGE should be positive.
4.	If a record contains ‘BR’ in FMTBEDRMS column then It should be either 1,2,3 or 4.
5.	FMTSTATUS should be same for all records.


In [4]:
# Define a function where error will be caught and code will not fail

def posnum(s):
    try:
        return int(s)
    except ValueError:
        return(-1)

In [5]:
# check bedroom critria

def bdrm(s):
    valid_list=['1','2','3','4']
    if s.find("BR") > 0 and s[s.find("BR")-1] not in  valid_list:
        return(-1)

In [6]:
# Check which CONTROL values are duplicates (more than 1)
dups=[ (k,v) for (k,v) in Counter(val_ds['CONTROL']).items() if v > 1  ]

In [7]:
# We have only one with 10 occurances
dups

[("'100013330103'", 10)]

In [8]:
dups_val =[ k for (k,v) in Counter(val_ds['CONTROL']).items() if v > 1  ]

In [9]:
dups_val

["'100013330103'"]

In [10]:
# Define the validation function

def validation():
    with open ('/Users/sandipayannandi/Downloads/egen/failed_records.csv', 'w',newline='') as f:
        writer=csv.writer(f)
        writer.writerow(list(val_ds.dtypes.index))
        for i in range(len(val_ds)):
            if ( posnum(val_ds['STRUCTURETYPE'][i]) <= 0 or val_ds['CONTROL'][i] in dups_val or val_ds['AGE1'][i] <=0 or bdrm(val_ds['FMTBEDRMS'][i])==-1  or  val_ds['FMTSTATUS'][i] !="'-5'"  )  :
                writer.writerow(list(val_ds.iloc[i]))
    

* Calling the validation() function will create the failed_records.csv csv file 

In [11]:
validation()

In [14]:
failed_records=pd.read_csv("/Users/sandipayannandi/Downloads/egen/failed_records.csv")

In [15]:
len(failed_records)

4455

* Glimpse of the failed records, for all records check the csv file generated above

In [16]:
failed_records[1:10]

,CONTROL,AGE1,METRO3,REGION,LMED,FMR,L30,L50,L80,IPOV,...,FMTINCRELFMRCAT,FMTCOST06RELAMICAT,FMTCOST08RELAMICAT,FMTCOST12RELAMICAT,FMTCOSTMEDRELAMICAT,FMTINCRELAMICAT,FMTASSISTED,FMTBURDEN,FMTREGION,FMTSTATUS
1,'100013330103',25,'1','2',64362,751,15429,25701,41108,15492,...,'3 GT FMR','4 60 - 80% AMI','4 60 - 80% AMI','4 60 - 80% AMI','4 60 - 80% AMI','6 100 - 120% AMI','0 Not Assisted','1 Less than 30%','-5','-5'
2,'100013330103',53,'3','2',64182,1002,17221,28686,45888,18126,...,'3 GT FMR','7 120% AMI +','7 120% AMI +','7 120% AMI +','6 100 - 120% AMI','7 120% AMI +','.','1 Less than 30%','-5','-5'
3,'100013330103',67,'1','2',64362,1239,13514,22496,35977,11057,...,'3 GT FMR','7 120% AMI +','7 120% AMI +','7 120% AMI +','6 100 - 120% AMI','7 120% AMI +','.','2 30% to 50%','-5','-5'
4,'100028130103',32,'1','3',56613,1349,20190,33623,53783,30834,...,'3 GT FMR','4 60 - 80% AMI','4 60 - 80% AMI','5 80 - 100% AMI','3 50 - 60% AMI','6 100 - 120% AMI','.','1 Less than 30%','-5','-5'
5,'100028170140',-9,'1','3',56613,868,12190,20301,32462,-6,...,'.','3 50 - 60% AMI','3 50 - 60% AMI','3 50 - 60% AMI','3 50 - 60% AMI','.','.','.','-5','-5'
6,'100102770145',-9,'2','3',77954,749,16434,27384,40379,-6,...,'.','7 120% AMI +','7 120% AMI +','7 120% AMI +','7 120% AMI +','.','.','.','-5','-5'
7,'100105130103',-9,'4','3',59547,589,12837,21389,34008,-6,...,'.','2 30 - 50% AMI','2 30 - 50% AMI','2 30 - 50% AMI','2 30 - 50% AMI','.','.','.','-5','-5'
8,'100013330103',39,'5','1',61917,1056,25706,42836,68005,49283,...,'2 50.1 - 100% FMR','7 120% AMI +','7 120% AMI +','7 120% AMI +','7 120% AMI +','2 30 - 50% AMI','.','3 50% or More','-5','-5'
9,'100130310140',-9,'2','3',57700,776,12150,20200,32350,-6,...,'.','1 LTE 30% AMI','1 LTE 30% AMI','1 LTE 30% AMI','1 LTE 30% AMI','.','.','.','-5','-5'


###  * Use case #3

* geopandas is the best way to get the shape file data, creates a pandas dtaframe

In [17]:
import geopandas as gp
file = "/Users/sandipayannandi/Downloads/egen/melbourne_accident.shp"

shp=gp.GeoDataFrame.from_file(file)


* In the above dtaframe the 'geometry' column is used to create two different columns 
'longitude' & 'latitude' at the end of the csv file

In [18]:
with open ('/Users/sandipayannandi/Downloads/egen/melbourne_accident_data.csv', 'w',newline='') as f:
    writer=csv.writer(f)
    writer.writerow(list(shp.dtypes.index)+['longitude', 'latitude'])
    for i in range(len(shp)):
            writer.writerow(list(shp.iloc[i]) + str(shp['geometry'][i]).replace('POINT (','').replace(')','').split())
    

* melbourne_accident_data.csv file should be checked for all the data and can be easily filtered by a latitude & longitude